In [1]:
%load_ext autoreload
%autoreload 2

In [41]:
import sys
sys.path.append('../')
import tokamakTK
from tokamakTK import get_ECT_regression

import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as sp
import statsmodels.api as sm
import matplotlib.patches as mpatches

from collections import Counter
from scipy.stats import f
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

pd.set_option('display.max_columns', None)
path = "../data/"
fig_path = "../../../LATEX/Latex Images/"

plt.rc('font',family = 'serif')
# ['IP', 'BT', 'NEL', 'PLTH', 'RGEO', 'KAREA', 'EPS', 'MEFF']
features = ['IP', 'NEL', 'PLTH', 'RGEO', 'EPS', 'MEFF']

In [42]:
# Obtained from Optimization

min_subset_ids_6357 = pd.read_csv(path+"R_ids_alpha_0.6357.csv")
min_subset_ids_9998 = pd.read_csv(path+"R_ids_alpha_0.9998.csv")

DB2 = pd.read_csv(path+"DB2P8.csv")
DB5 = pd.read_csv(path+"SELDB5_SVD.csv", low_memory=False) 

# Setting ELMy Dataset
DB5 = DB5[DB5["PHASE"].isin(['HGELM', 'HSELM', 'HGELMH', 'HSELMH'])]

# Removing Spherical TOKAMAKS
#DB5 = DB5[~DB5["TOK"].isin(['START','MAST','NSTX'])]

# There is two shots from DB2P8 missing in DB5
missing_shots = DB2[~DB2.id.isin( DB5.id.values )].reset_index(drop=True)
DB5 = pd.concat([DB5, missing_shots], axis=0, ignore_index=True)

# Labeling shots that had great impact in decreasing alpha_R
DB5.insert(loc=2,column="label_6357",value=[0]*len(DB5))
DB5.loc[(DB5[DB5.id.isin(min_subset_ids_6357.id)].index), "label_6357"] = 1

DB5.insert(loc=2,column="label_9998",value=[0]*len(DB5))
DB5.loc[(DB5[DB5.id.isin(min_subset_ids_9998.id)].index), "label_9998"] = 1

DB5.insert(0, 'intercept', np.ones(len(DB5)))


print(
    "  Subset that decrease alpha-R to 0.6357\n--------\n" +
    f"{ round( (len(min_subset_ids_6357)/len(DB5))*100     ,2)  }% affected alpha_R\n" + 
    f"{ round( (1 - len(min_subset_ids_6357)/len(DB5))*100 ,2)  }% did not affect alpha_R" +
    "\n\n\n  Subset that decrease alpha-R to 0.9998\n--------\n" +
    f"{ round( (len(min_subset_ids_9998)/len(DB5))*100     ,2)  }% affected alpha_R\n" + 
    f"{ round( (1 - len(min_subset_ids_9998)/len(DB5))*100 ,2)  }% did not affect alpha_R"
)

  Subset that decrease alpha-R to 0.6357
--------
23.45% affected alpha_R
76.55% did not affect alpha_R


  Subset that decrease alpha-R to 0.9998
--------
9.88% affected alpha_R
90.12% did not affect alpha_R


In [43]:
all_cases = [DB2.copy(),
             DB5.copy(),
             DB5[DB5.label_6357.isin([1]) | DB5.id.isin(DB2.id.values)],
             DB5[DB5.label_6357.isin([0])],
             DB5[DB5.label_9998.isin([1]) | DB5.id.isin(DB2.id.values)],
             DB5[DB5.label_9998.isin([0])]
]

def get_FPvalues(full_model, reduced_model):
    return (anova_lm(reduced_model, full_model)["F"][1], 
            anova_lm(reduced_model, full_model)["Pr(>F)"][1])

In [44]:
# Creating table with F-test and P-test values per case
TESTS = pd.DataFrame(np.zeros((len(features), len(all_cases)*2)), 
                    columns=['FV0', 'PV0', 'FV1', 'PV1', 'FV2', 'PV2', 'FV3', 'PV3', 'FV4', 'PV4', 'FV5', 'PV5'],
                    index=features)

In [45]:
for i, data_ in enumerate(all_cases):
    for f in features:
        
        reduced_features = features.copy()
        reduced_features.remove(f)
        
        # get_ECT_regression takes the regression in log-space with intercept
        
        full_regression = get_ECT_regression(data_)
        reduced_regression = get_ECT_regression(data_, features=reduced_features)
        
        TESTS.loc[f, f"FV{i}"] = np.round(get_FPvalues(full_regression, reduced_regression)[0],3)
        TESTS.loc[f, f"PV{i}"] = np.round(get_FPvalues(full_regression, reduced_regression)[1],3)
        
TESTS.index.rename("removed", inplace=True)
c = ["F_DB2", "P_DB2", "F_DB5", "P_DB5", "F_d0.64", "P_d0.64", # (d): decreasing; (u): unaffected
                                         "F_u0.64", "P_u0.64", "F_d0.99", "P_d0.99",
                                                               "F_u0.99", "P_u0.99"]
TESTS.columns = c

In [46]:
TESTS

,F_DB2,P_DB2,F_DB5,P_DB5,F_d0.64,P_d0.64,F_u0.64,P_u0.64,F_d0.99,P_d0.99,F_u0.99,P_u0.99
removed,,,,,,,,,,,,
IP,1249.466,0.0,3942.077,0.0,2882.359,0.0,3309.297,0.0,1835.543,0.0,3463.845,0.0
NEL,212.009,0.0,138.505,0.0,28.650,0.0,746.469,0.0,6.588,0.0,289.091,0.0
PLTH,949.402,0.0,3467.893,0.0,763.190,0.0,5056.318,0.0,585.624,0.0,3850.336,0.0
RGEO,715.764,0.0,1401.616,0.0,129.401,0.0,3007.085,0.0,180.144,0.0,1850.355,0.0
EPS,58.315,0.0,33.756,0.0,103.552,0.0,213.779,0.0,30.362,0.0,56.642,0.0
MEFF,63.486,0.0,99.423,0.0,9.639,0.0,229.478,0.0,12.948,0.0,131.721,0.0


In [47]:
#TESTS.to_csv(path+"ANOVA_tests.csv")

In [48]:
TESTS[TESTS.columns[:4]]

,F_DB2,P_DB2,F_DB5,P_DB5
removed,,,,
IP,1249.466,0.0,3942.077,0.0
NEL,212.009,0.0,138.505,0.0
PLTH,949.402,0.0,3467.893,0.0
RGEO,715.764,0.0,1401.616,0.0
EPS,58.315,0.0,33.756,0.0
MEFF,63.486,0.0,99.423,0.0


In [49]:
TESTS[TESTS.columns[4:]]

,F_d0.64,P_d0.64,F_u0.64,P_u0.64,F_d0.99,P_d0.99,F_u0.99,P_u0.99
removed,,,,,,,,
IP,2882.359,0.0,3309.297,0.0,1835.543,0.0,3463.845,0.0
NEL,28.650,0.0,746.469,0.0,6.588,0.0,289.091,0.0
PLTH,763.190,0.0,5056.318,0.0,585.624,0.0,3850.336,0.0
RGEO,129.401,0.0,3007.085,0.0,180.144,0.0,1850.355,0.0
EPS,103.552,0.0,213.779,0.0,30.362,0.0,56.642,0.0
MEFF,9.639,0.0,229.478,0.0,12.948,0.0,131.721,0.0
